# Imports

In [2]:
! git clone https://github.com/ai-forever/sage.git

Cloning into 'sage'...
remote: Enumerating objects: 171, done.
remote: Counting objects: 100% (171/171), done.
remote: Compressing objects: 100% (118/118), done.
remote: Total 171 (delta 63), reused 140 (delta 45), pack-reused 0
Receiving objects: 100% (171/171), 13.70 MiB | 2.86 MiB/s, done.
Resolving deltas: 100% (63/63), done.


In [3]:
cd sage

/content/sage


In [4]:
! pip install .
! pip install -r requirements.txt

Processing /content/sage
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 14.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 94.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 73.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 72.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.0 MB/s eta 0:00

Processing ./wheels/augmentex-1.0.3-py3-none-any.whl (from -r requirements.txt (line 1))


In [1]:
# Load functionality

import os
import torch
from sage.spelling_correction import T5ModelForSpellingCorruption, RuM2M100ModelForSpellingCorrection, AvailableCorrectors

# Quick tour [English]

In [ ]:
# Load corrector

corrector = T5ModelForSpellingCorruption.from_pretrained(AvailableCorrectors.ent5_large.value)

In [ ]:
# Place model on your favorite device

corrector.model.to(torch.device("cuda:0"));

## Generate correct texts

Simply, corrector's API have two methods that allow
you to generate correct text.

First, `correct()` method: use it when you have single sample.
You can also provide additional `prefix` argument if needed,
and `**generation_params` of your choice.

Apparent counterpart is `batch_correct` method.
As a name suggests, most useful when you've batch of texts to correct.
You may also provide `batch_size` parameter.

In [ ]:
# Imagine you have bunch of texts with broken spelling.

samples = [
    "So I think we would not be live if our ancestors did not develop siences and tecnologies.",
    "There are very successful politicians that have never tried somthing new.",
    "second , birds navigate by landmarks like river , coastlines , and moutains.",
    "Because of this , I prefer studying concepts and ideas more thad learnig facts."
]

In [ ]:
# Model's been trained with "grammar: " prefix.
# Don't forget to past `prefix` when calling corresponding methods.

result = corrector.correct(samples[0], prefix="grammar: ")

In [ ]:
print(result[0])

So I think we would not be alive if our ancestors did not develop sciences and technologies.


In [ ]:
batch_result = corrector.batch_correct(samples, batch_size=1, prefix="grammar: ")

In [ ]:
print(*batch_result, sep="\n")

['So I think we would not be alive if our ancestors did not develop sciences and technologies.']
['There are very successful politicians that have never tried something new.']
['second, birds navigate by landmarks like rivers, coastlines, and mountains.']
['Because of this, I prefer studying concepts and ideas more than learning facts.']


In [ ]:
# Try with bigger `batch_size`

batch_result = corrector.batch_correct(samples, batch_size=4, prefix="grammar: ")

In [ ]:
print(*batch_result[0], sep="\n")

So I think we would not be alive if our ancestors did not develop sciences and technologies.
There are very successful politicians that have never tried something new.
second, birds navigate by landmarks like rivers, coastlines, and mountains.
Because of this, I prefer studying concepts and ideas more than learning facts.


In [ ]:
# Experiment with different `**generation_params`

batch_result = corrector.batch_correct(
    samples, batch_size=1, prefix="grammar: ", num_return_sequences=2, do_sample=True, top_k=50, top_p=0.95)

In [ ]:
for elem in batch_result:
    print(*elem, sep="\n")
    print()

So I think we would not be alive if our ancestors did not develop sciences and technologies.
So I think we would not be alive if our ancestors did not develop sciences and technologies.

There are very successful politicians that have never tried something new.
There are very successful politicians that have never tried something new.

second, birds navigate by landmarks like rivers, coastlines, and mountains.
second, birds navigate by landmarks like rivers, coastlines, and mountains.

Because of this, I prefer studying concepts and ideas more than learning facts.
Because of this, I prefer studying concepts and ideas more than learning facts.



## Validation on JFLEG

You can call the evaluation on any dataset that is available
either on HF hub or localy.

Remember, it should be properly formatted.
Two text files: `sources.txt` and `corrections.txt` in one folder. If you prefer
single file, you may want to use `data.csv` with two columns `source` and `correction`.
Or just write down the correct name of dataset on HF hub.

In [ ]:
# Make sure to stay inside sage directory or change path to validation data

metrics = corrector.evaluate(
    os.path.join(os.getcwd(), "data", "example_data", "jfleg"), batch_size=32, prefix="grammar: ")

In [ ]:
print(metrics)

{'Precision': 83.43, 'Recall': 84.25, 'F1': 83.84}


# Quick tour [Russian]

- M2M100-1.2B
- M2M100-418M
- FredT5-large

In [6]:
# For Russian we have wider range of available models.
# P.S. ent5_large model corresponds to the English language, of course)

print(*["{}: {}".format(item.name, item.value) for item in AvailableCorrectors], sep="\n")

m2m100_1B: ai-forever/RuM2M100-1.2B
m2m100_418M: ai-forever/RuM2M100-418M
fred_large: ai-forever/FRED-T5-large-spell
ent5_large: ai-forever/T5-large-spell


#### a. FredT5-large

In [11]:
fred_corrector = T5ModelForSpellingCorruption.from_pretrained(AvailableCorrectors.fred_large.value)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
#fred_corrector.model.to(torch.device("cuda:0"))

In [12]:
# Make up some spoiled sentences

samples = [
    "прийдя в МГТУ я был удивлен никого необноружив там…",
    "Нащщот Чавеса разве что не соглашусь.",
    "Мошный лазер - в нерабочем состоянии - 350 кредиток.",
    "Ощушаю себя с ними монголойдом, я никогда так много не молчала как молчю тут, и не потому, что языковый баръер или еще что-то, просто коментариев нет"
]

In [17]:
for s in samples:
  result_fred = fred_corrector.correct(s, prefix="Исправь: ")
  print(result_fred)



  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:04<00:00,  4.34s/it]


['прийдя в МГТУ я был удивлен никого не обнаружив там.. «при']




  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:03<00:00,  3.78s/it]


['На счет Чавеса разве что не соглашусь. На счет']




  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:04<00:00,  4.47s/it]


['Мощный лазер - в нерабочем состоянии - 350 кредиток']




  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:12<00:00, 12.29s/it]

['Ощущаю себя с ними монголойдом, я никогда так много не молчала как молчу тут, и не потому, что языковый барьер или еще что-то, просто коментариев нет, просто ком']


In [18]:
result_fred = fred_corrector.batch_correct(
    samples, batch_size=1, prefix="Исправь: ", num_return_sequences=2, do_sample=True, top_k=50, top_p=0.95)



  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:07<00:21,  7.18s/it]

 50%|█████     | 2/4 [00:11<00:11,  5.62s/it]

 75%|███████▌  | 3/4 [00:15<00:04,  4.59s/it]

100%|██████████| 4/4 [00:25<00:00,  6.44s/it]


In [19]:
result_fred

[['прийдя в МГТУ я был удивлен никого необнаружив там.. «',
  'прийдя в МГТУ я был удивлен никого не обнаружив там…прой'],
 ['Но что от Чавеса разве что не соглашусь. Насчет',
  'На счет Чавеса разве что не соглашусь. Насчет Ча'],
 ['Мощный лазер - в нерабочем состоянии - 350 кредиток',
  'Мощный лазер - в нерабочем состоянии - 350 кредиток'],
 ['Ощущаю себя с ними монголойдом, я никогда так много не молчала как молчу тут, и не потому, что языковый барьер или еще что-то, просто коментариев нет, просто ком',
  'Ощущаю себя с ними монголом, я никогда так много не молчала как молчу тут, и не потому, что языковый барьер или еще что-то, просто коментариев нет, просто комента']]

In [4]:
mistakes  = [
    'ОТДЕЛ МИНИСТЕРСТВА ВНУТРЕННИХ ДЕЛ РОССИЙСКОЙ Ф РАЦИИ ПО РАЙОНУ ЛУЖНИКИ ГОРОДА МОСКВЫ',
    '(ОМВЛ России по району Лужники г.Москвы)',
    '08 НО! м оз 3773 на № от ОТЕЙ КЕ',
    '© предоставлении информации',
    'ул. Преспеиская набережная',
    'Марии Александронвны 21.02.1983 г.р. зарегистрированной но адрес Москва ул. Батырская д.17',
    'по факту совершения в отношение заявителя мошеннический действий',
    'расчетный” счет Идинадлежащей Константиновой Марии',
    'Исп: Маминский 8-965-342-. Р.Р.'
]

In [31]:
result_fred = fred_corrector.batch_correct(
    mistakes, batch_size=1, prefix="Исправь: ", num_return_sequences=3, do_sample=True, top_k=5, top_p=0.95)



  0%|          | 0/9 [00:00<?, ?it/s]

 11%|█         | 1/9 [00:14<01:57, 14.72s/it]

 22%|██▏       | 2/9 [00:19<01:01,  8.72s/it]

 33%|███▎      | 3/9 [00:24<00:42,  7.04s/it]

 44%|████▍     | 4/9 [00:25<00:23,  4.69s/it]

 56%|█████▌    | 5/9 [00:27<00:15,  3.87s/it]

 67%|██████▋   | 6/9 [00:36<00:16,  5.42s/it]

 78%|███████▊  | 7/9 [00:39<00:09,  4.62s/it]

 89%|████████▉ | 8/9 [00:42<00:04,  4.25s/it]

100%|██████████| 9/9 [00:47<00:00,  5.30s/it]


In [32]:
for i in range(len(result_fred)):
  r = result_fred[i]
  print(mistakes[i])
  print(r[0])
  print(r[1])
  print()

ОТДЕЛ МИНИСТЕРСТВА ВНУТРЕННИХ ДЕЛ РОССИЙСКОЙ Ф РАЦИИ ПО РАЙОНУ ЛУЖНИКИ ГОРОДА МОСКВЫ
ОТДЕЛ МИНИСТЕРСТВА ВНУТРЕННИХ ДЕЛ РОССИЙСКОЙ ФРАЦИИ ПО РАЙОНУ ЛУЖНИКИ ГОРОДА МОСКВЫ
ОТДЕЛ МИНИСТЕРСТВА ВНУТРЕННИХ ДЕЛ РОССИЙСКОЙ ФРАЦИИ ПО РАЙОНУ ЛУЖНИКИ ГОРОДА МОСКВЫ

(ОМВЛ России по району Лужники г.Москвы)
(ОМВД России по району Лужники г.Москвы) 
(ОМВД России по району Лужники г. Москвы)2 (

08 НО! м оз 3773 на № от ОТЕЙ КЕ
08 НОЯ! мз 3773 на номер от ОТЕЦК
08 НОЯ! номер 3773 на 9КЛИТЕЙКЕ

© предоставлении информации
@ предоставление информации
|О предоставлении

ул. Преспеиская набережная
ул. Пресненская набережная1
ул. Пречистенская набережная

Марии Александронвны 21.02.1983 г.р. зарегистрированной но адрес Москва ул. Батырская д.17
Марии Александровны 21.02.1983 г.р. зарегистрированной на адрес Москва ул. Батырская д.17а.
Марии Александровны 21.02.1983 г.р. зарегистрированной на адрес Москва ул. Батырская д.17.17

по факту совершения в отношение заявителя мошеннический действий
по факту соверш

In [33]:
text_full ='УВД по ЮВАО ГУ МВД России по г. Москве ОТДЕЛ МИНИСТЕРСТВА ВНУТРЕННИХ ДЕЛ РОССИЙСКОЙ Ф РАЦИИ ПО РАЙОНУ ЛУЖНИКИ ГОРОДА МОСКВЫ (ОМВЛ России по району Лужники г.Москвы) тел. Кузовская, д.12, Москва 159348 тел. (495) 454-65-94, факс (495) 554-87-56 08 НО! м оз 3773 на № от ОТЕЙ КЕ © предоставлении информации Руководителю ПАО «Банк ВТБ» г. Москва ул. Преспеиская набережная д. 12 = 5) В связи с проводимой проверкой в порядке ст. 144-145 УПК РФ (КУСИ № 21058 от 30.10.2019 года) по заявлению гр. Константиновой Марии Александронвны 21.02.1983 г.р. зарегистрированной но адрес Москва ул. Батырская д.17 кв.32, по факту совершения в отношение заявителя мошеннический действий. Руководствуясь п.4 п.29 ч.1 сет. 13 ФЗ РФ от 07.02.2017 года № 3-ФЗ «О полиции» п.2, ч.1 ст.б, 7 ФЗ «Об оперативно- розыскной деятельности» и ст.26 ФЗ «О банках и банковской деятельности» прошу Вас сообщить: в каком отделение банка ПАО «ВТБ» открыта и в настоящий момент обслуживается расчетный” счет Идинадлежащей Константиновой Марии Александронвны 21.02.1983 г.р. Врио начальника Мусатов Р.Д. Банк ВТБ (ПАО) 09. 01. 2020 Исп: Маминский 8-965-342-. Р.Р.'

In [34]:
fred_corrector.correct(text_full, prefix="Исправь: ")



  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [01:53<00:00, 113.06s/it]


['УВД по ЮВАО ГУ МВД России по г. Москве ОТДЕЛ МИНИСТЕРСТВА ВНУТРЕННИХ ДЕЛ РОССИЙСКОЙ ФРАЦИИ ПО РАЙОНУ ЛУЖНИКИ ГОРОДА МОСКВЫ (ОМВЛ России по району Лужники г.Москвы) тел. Кузовская, д.12, Москва 159348 тел. (495) 454-65-94, факс (495) 554-87-56 08 НО! м оз 3773 на No от ОТЕЙ КЕ @ предоставлении информации Руководителю ПАО «Банк ВТБ» г. Москва ул. Преспенская набережная д. 12 = 5) В связи с проводимой проверкой в порядке ст. 144-145 УПК РФ (КУСИ No 21058 от 30.10.2019 года) по заявлению гр. Константиновой Марии Александровны 21.02.1983 г.р. зарегистрированной на адрес Москва ул. Батырская д.17 кв.32, по факту совершения в отношение заявителя мошеннический действий. Руководствуясь п.4 п.29 ч.1 ст. 13 ФЗ РФ от 07.02.2017 года No 3-ФЗ «О полиции» п.2, ч.1 ст.б, 7 ФЗ «Об оперативно- розыскной деятельности» и ст.26 ФЗ «О банках и банковской деятельности» прошу Вас сообщить: в каком отделение банка ПАО «ВТБ» открыта и в настоящий момент обслуживается расчетный” счет Идинадлежащей Константинов

#### b. M2M100-1.2B

In [2]:
m2m_1b_corrector = RuM2M100ModelForSpellingCorrection.from_pretrained(AvailableCorrectors.m2m100_1B.value)

tokenizer_config.json:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/465k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/489k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

In [5]:
result_1b = m2m_1b_corrector.batch_correct(
    mistakes, batch_size=1, num_return_sequences=2, do_sample=True, top_k=50, top_p=0.95)

100%|██████████| 9/9 [01:43<00:00, 11.48s/it]


In [7]:
for i in range(len(result_1b)):
  r = result_1b[i]
  print(mistakes[i])
  print(r[0])
  print(r[1])
  print()

ОТДЕЛ МИНИСТЕРСТВА ВНУТРЕННИХ ДЕЛ РОССИЙСКОЙ Ф РАЦИИ ПО РАЙОНУ ЛУЖНИКИ ГОРОДА МОСКВЫ
ОТДЕЛ МИНИСТЕРСТВА ВНУТРЕННИХ ДЕЛ РОССИЙСКОЙ ФРАЦИИ ПО РАЙОНУ ЛУЖНИКИ ГОРОДА МОСКВЫ
ОТДЕЛ МИНИСТЕРСТВА ВНУТРЕННИХ ДЕЛ РОССИЙСКОЙ ФРАЦИИ ПО РАЙОНУ ЛУЖНИКИ ГОРОДА МОСКВЫ

(ОМВЛ России по району Лужники г.Москвы)
(ОМВЛ России по району Лужники г.Москвы)
(ОМВЛ России по району Лужники г.Москвы)

08 НО! м оз 3773 на № от ОТЕЙ КЕ
08 НО! мост 3773 на No1 от АТЕЙКЕ
08 НО! мост 3773 на No7 от ТЕЛЕЙКЕ

© предоставлении информации
предоставлении информации
предоставлении информации

ул. Преспеиская набережная
ул. Пресненская набережная
ул. Пресненская набережная

Марии Александронвны 21.02.1983 г.р. зарегистрированной но адрес Москва ул. Батырская д.17
Марии Александровны 21.02.1983 г.р. зарегистрированной на адрес Москва ул. Батырская д.17
Марии Александровны 21.02.1983 г.р. зарегистрированной на адрес Москва ул. Батырская д.17

по факту совершения в отношение заявителя мошеннический действий
по факту совершения

In [ ]:
# Load corrector

# NOTE: all three models may exceed the amount of RAM available in free version of Colab.
# If the case, comment out one or two models and make sure to comment corresponding outputs and samples.

# m2m_1b_corrector = RuM2M100ModelForSpellingCorrection.from_pretrained(AvailableCorrectors.m2m100_1B.value)
# m2m_418m_corrector = RuM2M100ModelForSpellingCorrection.from_pretrained(AvailableCorrectors.m2m100_418M.value)
fred_corrector = T5ModelForSpellingCorruption.from_pretrained(AvailableCorrectors.fred_large.value)

In [ ]:
# result_1b = m2m_1b_corrector.correct(samples[0])
# result_418m = m2m_418m_corrector.correct(samples[0])
result_fred = fred_corrector.correct(samples[0], prefix="Исправь: ")

In [ ]:
print("m2m1b")
print(result_1b[0])
print()

print("m2m418m")
print(result_418m[0])
print()

print("fred")
print(result_fred[0])

m2m1b
прийдя в МГТУ я был удивлен никого не обнаружив там...

m2m418m
Прийдя в МГТУ, я был удивлен, никого не обнаружив там...

fred
прийдя в МГТУ я был удивлен никого не обнаружив там.. «при


In [ ]:
result_1b = m2m_1b_corrector.batch_correct(samples, batch_size=1)
result_418m = m2m_418m_corrector.batch_correct(samples, batch_size=1)
result_fred = fred_corrector.batch_correct(samples, prefix="Исправь: ", batch_size=1)

In [ ]:
print("m2m1b")
print(*result_1b, sep="\n")
print()

print("m2m418m")
print(*result_418m, sep="\n")
print()

print("fred")
print(*result_fred, sep="\n")

m2m1b
['прийдя в МГТУ я был удивлен никого не обнаружив там...']
['Насчет Чавеса разве что не соглашусь.']
['Мощный лазер - в нерабочем состоянии - 350 кредиток.']
['Ощущаю себя с ними монголойдом, я никогда так много не молчала как молчу тут, и не потому, что языковый барьер или еще что-то, просто комментариев нет']

m2m418m
['Прийдя в МГТУ, я был удивлен, никого не обнаружив там...']
['Нащ от Чавеса. Разве что не соглашусь...']
['Мощный лазер - в нерабочем состоянии - 350 кредиток.']
['Ощушаю себя с ними монголойдом. Я никогда так много не молчала, как молчаю тут. И не потому, что языковый баръер или еще что-то, просто комментариев нет.']

fred
['прийдя в МГТУ я был удивлен никого не обнаружив там.. «при']
['На счет Чавеса разве что не соглашусь. На счет']
['Мощный лазер - в нерабочем состоянии - 350 кредиток']
['Ощущаю себя с ними монголойдом, я никогда так много не молчала как молчу тут, и не потому, что языковый барьер или еще что-то, просто коментариев нет, просто ком']


In [ ]:
result_1b = m2m_1b_corrector.batch_correct(
    samples, batch_size=1, num_return_sequences=2, do_sample=True, top_k=50, top_p=0.95)
result_418m = m2m_418m_corrector.batch_correct(
    samples, batch_size=1, num_return_sequences=2, do_sample=True, top_k=50, top_p=0.95)
result_fred = fred_corrector.batch_correct(
    samples, batch_size=1, prefix="Исправь: ", num_return_sequences=2, do_sample=True, top_k=50, top_p=0.95)

In [ ]:
sep = "\n------------------------------------------------------------------------------------------------------------\n"
result_1b = [elem[0] + "\n" + elem[1] for elem in result_1b]
result_418m = [elem[0] + "\n" + elem[1] for elem in result_418m]
result_fred = [elem[0] + "\n" + elem[1] for elem in result_fred]

In [ ]:
print("m2m1b")
print(*result_1b, sep=sep)
print()

print("m2m418m")
print(*result_418m, sep=sep)
print()

print("fred")
print(*result_fred, sep=sep)
print()

m2m1b
прийдя в МГТУ я был удивлен никого не обнаружив там...
прийдя в МГТУ я был удивлен никого не обнаружив там...
------------------------------------------------------------------------------------------------------------
Насчет Чавеса разве что не соглашусь.
Насчет Чавеса разве что не соглашусь.
------------------------------------------------------------------------------------------------------------
Мощный лазер - в нерабочем состоянии - 350 кредиток.
Мощный лазер - в нерабочем состоянии - 350 кредиток.
------------------------------------------------------------------------------------------------------------
Ощущаю себя с ними монголойдом, я никогда так много не молчала как молчу тут, и не потому, что языковый барьер или еще что-то, просто комментариев нет
Ощущаю себя с ними монголойдом, я никогда так много не молчала как молчу тут, и не потому, что языковый барьер или еще что-то, просто комментариев нет

m2m418m
Прийдя в МГТУ, я был удивлен, никого не обнаружив там...
Прийдя 

## Validation

In [ ]:
# Load available datasets

from sage.utils import DatasetsAvailable

In [ ]:
# Available datasets at HF hub

print(*["{}: {}".format(item.name, item.value) for item in DatasetsAvailable], sep="\n")

MultidomainGold: Multidomain gold dataset. For more see `ai-forever/spellcheck_benchmark`.
RUSpellRU: Social media texts and blogs. For more see `ai-forever/spellcheck_benchmark`.
MedSpellchecker: Medical anamnesis. For more see `ai-forever/spellcheck_benchmark`.
GitHubTypoCorpusRu: Github commits. For more see `ai-forever/spellcheck_benchmark`.


### M2M100-1.2B

In [ ]:
# Place model on device

m2m_1b_corrector.model.to(torch.device("cuda:0"));

In [ ]:
metrics = m2m_1b_corrector.evaluate("RUSpellRU", batch_size=32)

In [ ]:
print("m2m1b RUSpellRU:")
print(metrics)

m2m1b RUSpellRU:
{'Precision': 59.44, 'Recall': 43.32, 'F1': 50.12}


### M2M100-418M

In [ ]:
# Place model on device

m2m_418m_corrector.model.to(torch.device("cuda:0"));

In [ ]:
metrics = m2m_418m_corrector.evaluate("MultidomainGold", batch_size=16)

In [ ]:
print("m2m418m MultidomainGold:")
print(metrics)

m2m418m MultidomainGold:
{'Precision': 32.82, 'Recall': 57.69, 'F1': 41.84}


### FredT5-large

In [ ]:
# Place model on device

fred_corrector.model.to(torch.device("cuda:0"));

In [ ]:
metrics = fred_corrector.evaluate("GitHubTypoCorpusRu", prefix="Исправь: ", batch_size=1)

In [ ]:
print("fred GitHubTypoCorpusRu:")
print(metrics)

fred GitHubTypoCorpusRu:
{'Precision': 52.73, 'Recall': 41.75, 'F1': 46.6}
